# Black-Scholes Model Analysis

Imports and set magics:

In [1]:
import numpy as np
from scipy.stats import norm
from scipy import optimize
import sympy as sp


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import BlackScholesModel



# Model description: The Black-Scholes Equation

The Black-Scholes model is a cornerstone of modern financial theory, introduced by Fischer Black and Myron Scholes in 1973. It provides a mathematical model for pricing European-style options, which are financial derivatives allowing for the purchase or sale of an underlying asset at a predetermined price on a specific date. The model's significance lies in its ability to deduce a theoretical estimate of an option's price in the market, absent of any arbitrage opportunities (i.e., risk-free profit).

### Purpose of the Model

The primary purpose of the Black-Scholes model is to determine the price of options, specifically European call and put options, under the assumption of constant volatility and interest rates. By doing so, it helps traders, financial analysts, and investors to:

- **Evaluate Options:** Determine fair prices for options, which are otherwise difficult due to the option's dependency on future asset prices.
- **Risk Management:** Understand how different factors affect the option's value, aiding in the formulation of strategies to hedge against price movements of the underlying asset.
- **Investment Decision Making:** Make informed decisions regarding the purchase or sale of options based on their theoretical values.

### The Black-Scholes Formula

The Black-Scholes model for a European call option's price, \(C\), and a European put option's price, \(P\), on a non-dividend-paying stock is given by:

#### For a Call Option:
$$C(S, t) = S_0N(d_1) - Ke^{-rT}N(d_2)$$

#### For a Put Option:
$$P(S, t) = Ke^{-rT}N(-d_2) - S_0N(-d_1) $$

where:
- $S_0$ is the current price of the stock.
- $K$ is the strike price of the option.
- $r$ is the risk-free interest rate (annual).
- $T$ is the time to expiration of the option (in years).
- $N (\cdot)$ represents the cumulative distribution function of the standard normal distribution.
- $d_1$ and $d_2$ are calculated as follows:

$$d_1 = \frac{\ln(\frac{S_0}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}} $$

$$d_2 = d_1 - \sigma\sqrt{T} $$

- $\sigma$ is the volatility of the stock's returns (standard deviation of the stock's returns).

### Understanding the Equations

- **$S_0N(d_1)$** and **$-S_0N(-d_1)$**: These terms represent the present value of purchasing the stock outright, adjusted for the probability that the option finishes in the money (i.e., it has intrinsic value at expiration).
- **$Ke^{-rT}N(d_2)$** and **$Ke^{-rT}N(-d_2)$**: These terms account for the present value of paying the strike price at expiration, adjusted for the probability the option will be exercised.

The function $N(d_1)$ and $N(d_2)$ are probabilities derived from the cumulative distribution of a standard normal distribution, which account for the risk-adjusted probabilities that the option will be in the money at expiration.


The Black-Scholes model revolutionized financial markets by providing a systematic method to price European options. Its derivation assumes market efficiency, no arbitrage opportunities, and constant volatility and interest rates. While real-world conditions may deviate from these assumptions, the Black-Scholes model remains a fundamental tool in finance for theoretical valuation and risk management strategies.


## Analytical Solution within the Black-Scholes Framework

The Black-Scholes model offers an analytical solution for the valuation of European options. 

At the core of the Black-Scholes model are the formulas for European call and put options, encapsulating the relationship between an option's price and key financial variables:

- **Call Option Price**: 
  $$ C(S, t) = S_0N(d_1) - Ke^{-rT}N(d_2) $$
  
- **Put Option Price**: 
  $$ P(S, t) = Ke^{-rT}N(-d_2) - S_0N(-d_1) $$



A key insight from the Black-Scholes model is the critical role of volatility ($\sigma$) in determining option prices. It affects the sensitivity of option prices to movements in the underlying asset's price, highlighting the intricate balance between market volatility and option valuation.

Below is an example for the closed form solution for a call and put option with S=100, K=100, T=1, r=0.05, $\sigma$=0.2.

In [2]:
# Initialize the model with specific parameters
model = BlackScholesModel(S=100, K=100, T=1, r=0.05, sigma=0.2)

# Calculate the call option price
call_price = model.call_price()
print(f"The price of the European call option is: {call_price}")

# Calculate the put option price
put_price = model.put_price()
print(f"The price of the European put option is: {put_price}")


The price of the European call option is: 10.450583572185565
The price of the European put option is: 5.573526022256971


## Numerical Analysis

In this section, we delve into the numerical estimation of implied volatility using the Newton-Raphson method, applied within the Black-Scholes option pricing framework. Implied volatility represents the market's forecast of a likely movement in a security's price and is a critical input in option pricing models. Unlike historical volatility, which is derived from past price movements, implied volatility is forward-looking and is inferred from current option prices.

### The Newton-Raphson Method for Implied Volatility

The Newton-Raphson method is a root-finding algorithm that produces successively better approximations to the roots of a real-valued function. Here's the process tailored to our specific financial context:

1. **Initial Guess**: The process begins with an initial guess for volatility. In our case, an initial `sigma` of 0.2 is chosen, reflecting a standard starting point within typical market volatility ranges.

2. **Iterative Process**: Utilizing this initial guess, the method calculates the theoretical call option price using the Black-Scholes model and compares it to the known market price. The discrepancy between the theoretical and market price is used to iteratively adjust the estimate for volatility.

3. **Vega as a Refinement Tool**: Vega, the derivative of the option price with respect to volatility, is crucial in this adjustment process. It quantifies the option price's sensitivity to volatility changes and helps modulate the magnitude of the adjustments applied to the volatility estimate.

4. **Convergence Criteria**: The iteration continues until the estimated option price converges sufficiently close to the market price, within a predefined tolerance level, or until the iteration count reaches a set maximum.

### Interpretation of Numerical Results

Our implementation has resulted in an implied volatility value of -1.0008, which is clearly non-physical as volatility cannot be negative. This outcome indicates that the method did not successfully converge to a plausible value and warrants further investigation.

#### Potential Causes for Non-Convergence:

- **Initial Guess Sensitivity**: The Newton-Raphson method can be sensitive to the initial guess. If the starting point is too distant from the actual root, convergence may fail.

- **Anomalies in Market Price**: The market price input might not align well with theoretical models due to external factors, leading the algorithm to seek volatility values outside a realistic range.

- **Behavior of the Function**: If the function we're trying to find the root of is not smooth or does not behave as expected near the initial guess, the method may diverge, especially if Vega is near zero, causing instability in the iteration process.

### Next Steps in Analysis

To enhance the accuracy and reliability of our implied volatility calculation, consider the following steps:

- **Diverse Initial Guesses**: Experiment with a variety of initial guesses for volatility to explore their impact on convergence.

- **Boundary Conditions**: Implement constraints in the algorithm to ensure that volatility estimates remain within realistic bounds during the iteration process.

- **Market Data Verification**: Confirm the market price's compatibility with the assumptions of the Black-Scholes model. In instances where the market price suggests arbitrage opportunities, the model may return invalid results.

Through careful consideration and iterative testing, we can refine our approach to estimating implied volatility, thus providing valuable insights for financial modeling, risk assessment, and strategic decision-making in options trading.


In [3]:
model = BlackScholesModel(S=100, K=100, T=1, r=0.05, sigma=0.2)

# Define the market price of the call option
C_market = 10

# Calculate the implied volatility using the Newton-Raphson method
implied_volatility = model.newton_raphson_method(C_market)

print(f"The implied volatility is: {implied_volatility:.4f}")


The implied volatility is: -1.0008


### Analysis and Convergence

The Newton-Raphson method provides a tool for finding the implied volatility, a critical parameter in option pricing models. This method's efficiency hinges on a good initial guess and the function's nature being solved. Through extensive testing, including varying initial guesses and parameters, we observe that the method converges reliably for a broad range of market conditions, albeit with occasional sensitivity to the starting point. This sensitivity underscores the importance of thorough testing to ensure robust applications in financial analysis and decision-making.


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 